In [1]:
from auxiliary import loadData_staticTargetAddrMatch, prepare_data_loaders
from algorithms import RssPosAlgo_NeuralNet_MLP4layer
from algorithms import RssPosAlgo_NearestNeighbour_Interpolation
from algorithms import RssPosAlgo_NearestNeighbour_GetKmeansDb
import numpy as np
import torch

### load model and dataloaders

In [2]:
datajsonpath = "../experiments/exp001_20240507_dorm_calpnp_converted_data/data-tshark/data.json"
inp_rss_vals, gt_locations = loadData_staticTargetAddrMatch(datajsonpath, second_hold = 5, shuffle=False, 
                                                            target_addresses=["d8:84:66:39:29:e8", "d8:84:66:4a:06:c9", "d8:84:66:03:61:00"])

epochs           = 601
batch_size       = 8
train_test_split = 0.5

train_loader, test_loader, xtr, ytr, xts, yts = prepare_data_loaders(inp_rss_vals, gt_locations, 
                                                                     batch_size = 32, 
                                                                     train_test_split = train_test_split)

print("Subset sizes | train:", xtr.shape[0], ", test:",xts.shape[0])

Subset sizes | train: 177 , test: 178


In [3]:
exp001_MLP = RssPosAlgo_NeuralNet_MLP4layer()
exp001_MLP.load_state_dict(torch.load('savedmodels/dev001_example_exp001_MLP4layer.pth'));
exp001_MLP.eval();

### define fusion algorithm

In [4]:
# simple mean
def RssPosAlgo_NeuralNet_NeNe_fusion(pred_nene, pred_nn):
    return (pred_nn.detach().numpy() + np.asarray(pred_nene))/2.0

### evaluate performance

In [5]:
exp001_nearestneighbour_db_kmeans = RssPosAlgo_NearestNeighbour_GetKmeansDb(xtr, ytr, num_clusters=3)
meanerror_nene_interp = 0;
meanerror_mlp         = 0;
meanerror_fusion      = 0;
for test_idx, x_test_sample in enumerate(xts): 
    loc_pred_mlp           = exp001_MLP(x_test_sample)
    loc_pred_nene_interp   = RssPosAlgo_NearestNeighbour_Interpolation(x_test_sample, exp001_nearestneighbour_db_kmeans)
    loc_fusion             = RssPosAlgo_NeuralNet_NeNe_fusion(loc_pred_nene_interp, loc_pred_mlp)
    meanerror_mlp         += (yts[test_idx].numpy() - loc_pred_mlp.detach().numpy())**2
    meanerror_nene_interp += (yts[test_idx].numpy() - loc_pred_nene_interp)**2
    meanerror_fusion      += (yts[test_idx].numpy() - loc_fusion)**2
print("MLP         :", np.linalg.norm(meanerror_mlp/(test_idx+1)))
print("NeNe+Interp :", np.linalg.norm(meanerror_nene_interp/(test_idx+1)))
print("Fusion      :", np.linalg.norm(meanerror_fusion/(test_idx+1)))

MLP         : 24.510115
NeNe+Interp : 25.331285817128602
Fusion      : 24.214751388593534
